<a href="https://colab.research.google.com/github/kshitij9969/Indicator_of_sucess_dataset/blob/main/Indicator_of_sucess_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [ ]:
# To do: 
# Two types of data needs to be extracted
# 1. Indexes
# 2. True values

# 1. Cost of living index by city
# 2. Cost of living index by country
# 3. Property price index
# 4. Quality of life index
# 5. Crime index
# 6. Health care index
# 7. Pollution index
# 8. Traffic index
# 9. 


# 3. Cost of living estimator by city
# 4. Food prices by city
# 5. Food prices by country
# 6. Prices of other commodity by city and country
# 7. Basket of prices by city
# 8. Taxi fare by city
# 9. Gas prices by city


# Cost of living index

### Cost of living index over the years. 

In [ ]:
response = requests.get("https://www.numbeo.com/cost-of-living/rankings.jsp")

In [ ]:
df = pd.read_html(response.text)

In [ ]:
df

In [ ]:
soup = BeautifulSoup(response.text, 'html.parser')

In [ ]:
table = soup.find("table", attrs={"id": "t2"})

In [ ]:
df = pd.read_html(str(table))[0]

In [ ]:
# Cleaning columns

for col in df.columns:
    df.rename(columns={col: "_".join(col.split(" "))}, inplace=True)


In [ ]:
# Split city and country

df['Country'] = df['City'].apply(lambda x: x.split(", ")[-1].strip())

In [ ]:
df['City'] = df['City'].apply(lambda x: x.split(", ")[0].strip())

In [ ]:
df.reset_index(inplace=True)

In [ ]:
df['Rank'] = df['index']

In [ ]:
df.drop(columns=['index'], inplace=True)

In [ ]:
df['Rank'] = df['Rank'].apply(lambda x: x + 1)

In [ ]:
df['Year'] = 2021
df['Month'] = 6
df['Day'] = 1

In [ ]:
df['Date'] = pd.to_datetime(df[['Year', 'Month', 'Day']])

In [ ]:
from requests.api import request
# Now collect for all other years and months

# Country links: 
links = [
         ("https://www.numbeo.com/cost-of-living/region_rankings.jsp?title=2021&region=002", "Africa"), # Africa
         ("https://www.numbeo.com/cost-of-living/region_rankings.jsp?title=2021&region=019", "America"), # America
         ("https://www.numbeo.com/cost-of-living/region_rankings.jsp?title=2021&region=142", "Asia"), # Asia
         ("https://www.numbeo.com/cost-of-living/region_rankings.jsp?title=2021&region=150", "Europe"), # Europe
         ("https://www.numbeo.com/cost-of-living/region_rankings.jsp?title=2021&region=009", "Oceania") # Oceania
]

# Collecting links for regions

final_dataframe = pd.DataFrame(columns=['Rank', 'City', 'Country', 
                                        'Continent', 'Region', 
                                        'Cost_of_Living_Index', 'Rent_Index',
                                        'Cost_of_Living_Plus_Rent_Index', 
                                        'Groceries_Index', 
                                        'Restaurant_Price_Index', 
                                        'Local_Purchasing_Power_Index', 'Year', 
                                        'Month', 'Day', 'Date'])

years_list = ['2021-mid', '2021', '2020-mid', '2020', '2019-mid', '2019', '2018-mid', '2018', '2017-mid', '2017', '2016-mid', '2016', '2015-mid', '2015', '2014-mid', '2014', '2013', '2012', '2011', '2010', '2009']

for (i, continent) in links:
    res = requests.get(i)
    soup = BeautifulSoup(res.text)
    try:
        spans = soup.find("div", attrs={"class": "select_region_links"}).find_all("span", attrs={"class": "nobreak"})
    except Exception as e:
        # Oceania doesn't have any regions
        print(e)
        print(i)
    print(spans)
    for span in spans:
        url = span.find("a", href=True)['href']
        resp = requests.get(url)
        soup1 = BeautifulSoup(resp.text)
        # years_list = [i['value'] for i in soup1.find_all("form", attrs={"class": "changePageForm"})[0].find("select").find_all("option")]
        # print(resp.text)
        table = soup.find("table", attrs={"id": "t2"})
        df = pd.read_html(str(table))[0]
        # Clean columns
        for col in df.columns:
            df.rename(columns={col: "_".join(col.split(" "))}, inplace=True)
        
        # Correct Rank
        df.reset_index(inplace=True)
        df['Rank'] = df['index']
        df['Rank'] = df['Rank'].apply(lambda x: x + 1)
        df.drop(columns=['index'], inplace=True)
        # Add month year and day
        df['Year'] = 2021
        df['Month'] = 1
        df['Day'] = 1

        df['Date'] = pd.to_datetime(df[['Year', 'Month', 'Day']])
        # Add country and continent
        df['Country'] = df['City'].apply(lambda x: x.split(",")[-1].strip())
        df['City'] = df['City'].apply(lambda x: x.split(",")[0].strip())
        df['Continent'] = continent
        df['Region'] = soup1.find("h1").text.split(":")[0].strip()
        print(url)
        print(years_list)
    break
        # print(url)
        # print(years_list)
        # region_name = soup1.find("h1").text.split(":")[0].strip()
        # print(region_name)






In [ ]:
from requests.api import request
# Now collect for all other years and months

# Country links: 
links = [
         ("https://www.numbeo.com/cost-of-living/region_rankings.jsp?title=year&region=002", "Africa"), # Africa
         ("https://www.numbeo.com/cost-of-living/region_rankings.jsp?title=year&region=019", "America"), # America
         ("https://www.numbeo.com/cost-of-living/region_rankings.jsp?title=year&region=142", "Asia"), # Asia
         ("https://www.numbeo.com/cost-of-living/region_rankings.jsp?title=year&region=150", "Europe"), # Europe
         ("https://www.numbeo.com/cost-of-living/region_rankings.jsp?title=year&region=009", "Oceania") # Oceania
]

# Collecting links for regions

final_dataframe = pd.DataFrame(columns=['Rank', 'City', 'Country', 
                                        'Continent', 'Region', 
                                        'Cost_of_Living_Index', 'Rent_Index',
                                        'Cost_of_Living_Plus_Rent_Index', 
                                        'Groceries_Index', 
                                        'Restaurant_Price_Index', 
                                        'Local_Purchasing_Power_Index', 'Year', 
                                        'Month', 'Day', 'Date'])

years_list = ['2021-mid', '2021', '2020-mid', '2020', '2019-mid', '2019', 
              '2018-mid', '2018', '2017-mid', '2017', '2016-mid', '2016', 
              '2015-mid', '2015', '2014-mid', '2014', '2013', '2012', 
              '2011', '2010', '2009']
url_list = []

for year in years_list:
    for (i, continent) in links:
        res = requests.get(i.replace("year", year))
        soup = BeautifulSoup(res.text)
        try:
            spans = soup.find("div", attrs={"class": "select_region_links"}).find_all("span", attrs={"class": "nobreak"})
        except Exception as e:
            # Oceania doesn't have any regions
            print(e)
            print(i)
        for span in spans:
            url = span.find("a", href=True)['href']
            url_list.append(url)
print(url_list)




In [ ]:
url_list

In [ ]:
final_dataframe = pd.DataFrame(columns=['Rank', 'City', 'Country', 
                                        'Continent', 'Region', 
                                        'Cost_of_Living_Index', 'Rent_Index',
                                        'Cost_of_Living_Plus_Rent_Index', 
                                        'Groceries_Index', 
                                        'Restaurant_Price_Index', 
                                        'Local_Purchasing_Power_Index', 'Year', 
                                        'Month', 'Day', 'Date', 'URL'])
for url in url_list:
    resp = requests.get(url)
    print(url)
    soup1 = BeautifulSoup(resp.text)
    # years_list = [i['value'] for i in soup1.find_all("form", attrs={"class": "changePageForm"})[0].find("select").find_all("option")]
    # print(resp.text)
    table = soup1.find("table", attrs={"id": "t2"})
    df = pd.read_html(str(table))[0]
    # Clean columns
    for col in df.columns:
        df.rename(columns={col: "_".join(col.split(" "))}, inplace=True)
    
    # Correct Rank
    df.reset_index(inplace=True)
    df['Rank'] = df['index']
    df['Rank'] = df['Rank'].apply(lambda x: x + 1)
    df.drop(columns=['index'], inplace=True)
    year = url.split("?")[-1].split("&")[0].split("=")[-1]
    if "mid" in year:
        year = int(year.split("-")[0].strip())
        month = 6
    else:
        year = int(year)
        month = 1
    # Add month year and day
    df['Year'] = year
    df['Month'] = month
    df['Day'] = 1

    df['Date'] = pd.to_datetime(df[['Year', 'Month', 'Day']])
    # Add country and continent
    df['Country'] = df['City'].apply(lambda x: x.split(",")[-1].strip())
    df['City'] = df['City'].apply(lambda x: x.split(",")[0].strip())
    df['Continent'] = continent
    df['Region'] = soup1.find("h1").text.split(":")[0].strip()
    df['URL'] = url
    # print(df)
    final_dataframe = final_dataframe.append(df)



In [ ]:
final_dataframe[final_dataframe.Year == 2015]

In [ ]:
# Correcting the Continent


def get_continent(x):
    if "Africa" in x:
        return "Africa"
    elif "America" in x:
        return "America"
    elif "Oceania" in x:
        return "Oceania"
    elif "Asia" in x:
        return "Asia"
    else:
        return "Europe"

final_dataframe['Continent'] = final_dataframe['Region'].apply(lambda x: get_continent(x))

In [ ]:
final_dataframe[(final_dataframe.City == "Aachen") & (final_dataframe.duplicated() == True)].sort_values(by="City")

In [ ]:
final_dataframe.reset_index(drop=True, inplace=True)

In [ ]:
final_dataframe.drop_duplicates(subset=['City', 'Country', 'Continent', 'Region', 'Year', 'Month', 'Day', 'Date'])

In [ ]:
final_dataframe.to_csv("Cost_of_living_index.csv", index=False)